In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/Table5_Data.csv'

In [3]:
data = pd.read_csv(source)

In [4]:
data

,DONOR,Donor,SECTOR,Sector,AIDTYPE,Aid type,AMOUNTTYPE,Amount type,TIME,Year,Value,Flags
0,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1971,1971,2.94,NaN
1,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1972,1972,4.25,NaN
2,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1973,1973,1.60,NaN
3,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1974,1974,1.98,NaN
4,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1975,1975,2.78,NaN
5,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1976,1976,0.65,NaN
6,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1977,1977,15.85,NaN
7,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1978,1978,18.50,NaN
8,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1979,1979,18.81,NaN
9,1,Austria,100,I. Social Infrastructure & Services,528,Total ODA,A,Current Prices (USD millions),1980,1980,20.33,NaN


In [11]:
data[['SECTOR', 'Sector']].drop_duplicates()

,SECTOR,Sector
0,100,I. Social Infrastructure & Services
44,110,I.1. Education
88,111,"I.1.a. Education, Level Unspecified"
108,112,I.1.b. Basic Education
128,113,I.1.c. Secondary Education
148,114,I.1.d. Post-Secondary Education
168,120,I.2. Health
210,121,"I.2.a. Health, General"
230,122,I.2.b. Basic Health
250,130,I.3. Population Pol./Progr. & Reproductive Health


In [9]:
data[data['Sector'] == 'Total (V+VI+VII+VIII+IX)']

,DONOR,Donor,SECTOR,Sector,AIDTYPE,Aid type,AMOUNTTYPE,Amount type,TIME,Year,Value,Flags
1263,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1967,1967,0.52,NaN
1264,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1968,1968,1.54,NaN
1265,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1969,1969,23.43,NaN
1266,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1970,1970,8.86,NaN
1267,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1971,1971,12.79,NaN
1268,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1972,1972,12.96,NaN
1269,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1973,1973,13.82,NaN
1270,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1974,1974,72.93,NaN
1271,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1975,1975,18.70,NaN
1272,1,Austria,1000,Total (V+VI+VII+VIII+IX),528,Total ODA,A,Current Prices (USD millions),1976,1976,30.04,NaN


In [12]:
# country

In [13]:
donor = data[['DONOR', 'Donor']].copy()

In [14]:
donor.columns = ['donor', 'name']

In [15]:
donor = donor.drop_duplicates()

In [17]:
donor.to_csv('../ddf--entities--donor.csv', index=False)

In [18]:
# sector

In [19]:
sector = data[['SECTOR', 'Sector']].drop_duplicates().copy()

In [20]:
sector.columns = ['sector', 'name']

In [22]:
sector.to_csv('../ddf--entities--sector.csv', index=False)

In [23]:
# concepts

In [24]:
cdf = pd.DataFrame([['total_oda', 'Total ODA', 'measure'], 
                    ['year', 'Year', 'time'],
                    ['name', 'Name', 'string'],
                    ['donor', 'Donor', 'entity_domain'],
                    ['sector', 'Sector', 'entity_domain']
                   ], columns=['concept', 'name', 'concept_type'])

In [25]:
cdf.to_csv('../ddf--concepts.csv', index=False)

In [26]:
# datapoints

In [27]:
dps = data[['DONOR', 'SECTOR', 'Year', 'Value']].copy()

In [29]:
dps.columns = ['donor', 'sector', 'year', 'total_oda']

In [32]:
dps.dropna().to_csv('../ddf--datapoints--total_oda--by--donor--sector--year.csv', index=False)

In [31]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
0,"donor,sector,year",total_oda,ddf--datapoints--total_oda--by--donor--sector-...
0,donor,name,ddf--entities--donor.csv
0,sector,name,ddf--entities--sector.csv
